# Create PySpark DataFrame

In [1]:
import os
import sys
import gc

In [38]:
%%html
<style>
table {float:left}
</style>

#  Environemnt Variables

## Hadoop

In [2]:
os.environ['HADOOP_CONF_DIR'] = "/opt/hadoop/hadoop-3.2.2/etc/hadoop"

In [3]:
%%bash
export HADOOP_CONF_DIR="/opt/hadoop/hadoop-3.2.2/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 5

capacity-scheduler.xml
configuration.xsl
container-executor.cfg
core-site.xml
core-site.xml.48132.2022-02-15@12:29:41~


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [4]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip",
    "/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
])

# Data

* [The UC Irvine Machine Learning Repository  - Record Linkage Comparison Patterns Data Set](https://archive.ics.uci.edu/ml/datasets/Record+Linkage+Comparison+Patterns)

In [26]:
%%bash
mkdir -p data/linkage
cd data/linkage/
curl -L -o donation.zip https://bit.ly/1Aoywaq

unzip -f donation.zip
unzip -f 'block_*.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0    529      0 --:--:-- --:--:-- --:--:--   529
100 53.8M  100 53.8M    0     0  7635k      0  0:00:07  0:00:07 --:--:-- 10.3M


Archive:  donation.zip
Archive:  block_9.zip

Archive:  block_4.zip

Archive:  block_10.zip

Archive:  block_1.zip

Archive:  block_6.zip

Archive:  block_5.zip

Archive:  block_2.zip

Archive:  block_8.zip

Archive:  block_3.zip

Archive:  block_7.zip



10 archives were successfully processed.


In [29]:
%%bash
cd data/linkage/
hdfs dfs -mkdir -p linkage
hdfs dfs -put -f block_*.csv linkage

---
# Spark Session


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

2022-02-19 07:31:33,602 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-19 07:31:36,147 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


# Read CSV

* [SparkSQL CSV Files](https://spark.apache.org/docs/latest/sql-data-sources-csv.html)

> Spark SQL provides spark.read().csv("file_name") to read a file or directory of files in CSV format into Spark DataFrame, and dataframe.write().csv("path") to write to a CSV file. Function option() can be used to customize the behavior of reading or writing.

[SparkSession.read()](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/SparkSession.html#read--) returns [DataFrameReader](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrameReader.html) instance which has [option](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrameReader.html#option-java.lang.String-boolean-) method by which we can specify CSV options.

The options are listed in [Data Source Option](https://spark.apache.org/docs/latest/sql-data-sources-csv.html#data-source-option)

In [36]:
prev = spark.read.csv("linkage")
prev.printSchema()
del prev

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



## CSV Options

* Schema Inference 
* Null value replacement
* Header handling

In [35]:
parsed = spark.read\
    .option("header", "true")\
    .option("nullValue", "?")\
    .option("inferSchema", "true")\
    .csv("linkage")

parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [37]:
parsed.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

# DataFrame Structure

## Compasison with HTML

| HTML       | HTML Description             | Spark       | Spark Description                                                                                                                                                                                                                                                                                                                       |   |
|------------|-------------------------|-------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---|
| ```<table>```    | List of ```<tr>```            | DataFrame   | List of Row, and DataFrame is an alias of type DataSet[Row].                                                                                                                                                                                                                                                                      |   |
| ```<tr>```       | Table row. List of ```<td> ```| Row         | List of typed values.  ```Row``` has the schema of type **StructType** |   |
|            |                         | StructType  | Defines the type of a Row and a list of StructField that defines a field of a Row.                                                                                                                                                                                                                                                |   |
|```<td>```Table  | Table field             | StructField | Specify the DataType of a field of a row.                                                                                                                                                                                                                                                                                         |   |
|            |                         | DataType    |                                                                                                                                                                                                                                                                                                                                   |   |


## DataFrame Schema Definition

Instead of using **Schema Inference**, you can provide the **Schema Definition** for  ```SparkSession.read.schema(schema).csv(file)``` to use when reading the CSV file.

In [44]:
from pyspark.sql.types import *
schema = StructType([
    StructField("id_1", IntegerType(), False),
    StructField("id_2", StringType(), False),
    StructField("cmp_fname_c1", DoubleType(), False)
])

for element in schema:
    print(element)
    
# spark.read.schema(schema).csv("...")

StructField(id_1,IntegerType,false)
StructField(id_2,StringType,false)
StructField(cmp_fname_c1,DoubleType,false)


## Row

Each row of the DataFrame is an instance of ```pyspark.sql.Row```.

In [56]:
import pyspark.sql 

row: pyspark.sql.Row = parsed.first()
row

Row(id_1=3148, id_2=8326, cmp_fname_c1=1.0, cmp_fname_c2=None, cmp_lname_c1=1.0, cmp_lname_c2=None, cmp_sex=1, cmp_bd=1, cmp_bm=1, cmp_by=1, cmp_plz=1, is_match=True)

## Column


In [58]:
id_1 = row['id_1']
print(f"id_1 type:{type(id_1)} value: {id_1}")

id_1 type:<class 'int'> value: 3148


# Stop Spark Session

In [8]:
spark.stop()



# Cleanup

In [9]:
del spark
gc.collect()

84